# 미세먼지 데이터 전처리

In [1]:
import pandas
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import pathlib
def read_csv_infolder(source_folder):
    csv_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    new_list = []
    for i,v in enumerate(csv_files):
        new_dict ={}
        if 'txt' in v:
            new_dict['sub'] = pd.read_csv(r'{}\{}'.format(source_folder,v),sep = '\t',encoding = 'utf-8')
            new_dict['title'] = v
            new_list.append(new_dict)
        elif 'csv' in v:
            new_dict['sub'] = pd.read_csv(r'{}\{}'.format(source_folder,v),sep = '|',encoding = 'utf-8')
            new_dict['title'] = v
            new_list.append(new_dict)
    new_df = pd.DataFrame(new_list)
    return new_df

### 종로구 데이터 전처리

In [2]:
### 경로를 지정
title = read_csv_infolder(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구')
### 저장된 파일이 있을시에 삭제
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_원본.csv')
if os.path.isfile(file):
    os.remove(file)
### 파일들을 하나로 합치기 
for idx in range(0, len(title.title)):
    temp = title.title[idx]
    v1 = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구\{}'.format(temp),sep=',',index_col=10)
    if file.exists() == False:
        v1.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_원본.csv', mode = 'a')
    else:
        v1.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_원본.csv', mode = 'a', header=False)

In [3]:
### 파일 불러오기
df = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_원본.csv',index_col=0)

C:\Users\dty\.conda\envs\June\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
###전처리(결측치 제거)
df= df.drop(columns = 'co2').drop(columns = 'vocs').drop(columns = 'flag')
df['pm25'] = pd.to_numeric(df.pm25)
df['pm10'] = pd.to_numeric(df.pm10)
df['tm'] = pd.to_numeric(df.tm)
df['noise'] = pd.to_numeric(df.noise)
df['humi'] = pd.to_numeric(df.humi)
df['temp'] = pd.to_numeric(df.temp)
df = df[df['noise'] != -9999]
df = df[df['temp'] != -9999.0]
df = df[df['humi'] != -9999]
df = df[df['pm25'] != -999]
df = df[df['pm10'] != -999]
df = df[df.pm25 <= 3000]  #3000 보다 큰 값들이 앞뒤의 값들과 차이가 크기 때문에 이상치로 판단
df = df.reset_index(drop=True)

### 시간별로 평균값 구하기

In [5]:
def date_h_cuter(dates):
    datelist=[]
    for idx in dates:
        date = str(idx)
        datelist.append(date[:10])
    return datelist

In [6]:
df['date_hourly'] = date_h_cuter(df.tm)
df['date_hourly'] = pd.to_datetime(df['date_hourly'], format='%Y%m%d%H', errors='ignore')
df.index = df.date_hourly
df = df.sort_index()

In [7]:
df = df.drop(columns='date_hourly')

In [8]:
###seiral 시간별 평균값 
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_pm_hourly.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in df.serial.unique():
    df_mean = df[df['serial'] == '{}'.format(idx)].reset_index().groupby('date_hourly').mean().reset_index().copy().drop(columns = 'tm')
    df_mean['serial'] = idx
    if file.exists() == False:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_pm_hourly.csv', mode = 'a')
    else:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_pm_hourly.csv', mode = 'a', header=False)

In [9]:
data_hourly = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_pm_hourly.csv', index_col=0).reset_index(drop=True)

In [10]:
station ="""V10O1610252
V10O1610546
V10O1610540
V10O1610542
V10O1610543
V10O1610544
V10O1610545
V10O1610567
V01o1610468
V10O1611289
V10O1611172
V10O1611634
V10O1611887
V10O1611639
V10O1611658
V10O1612113
V10O1611151
V10O1611145
V10O1611623
V10O1611750
V10O1611170
V10O1611684
V10O1611220
V10O1612106
V10O1611251
V10O1611173
V10O1611258
V10O1611255
V10O1611698
V10O1611722
V10O1611645"""
station= station.split('\n')

In [11]:
station_kor ="""교남동
종로 1,2,3,4가동
종로 1,2,3,4가동
종로 1,2,3,4가동
종로 1,2,3,4가동
종로 1,2,3,4가동
종로 1,2,3,4가동
종로 1,2,3,4가동
사직동
천연동
사직동
가회동
명동
종로5,6가동
이화동
종로 1,2,3,4가동
창신1동
종로 1,2,3,4가동
혜화동
종로 1,2,3,4가동
부암동
종로 1,2,3,4가동
창신1동
숭인2동
창신3동
종로 1,2,3,4가동
평창동
청운효자동
가회동
가회동
혜화동"""
station_kor= station_kor.split('\n')

In [12]:
station_with_names = []
for idx in range(0, len(station)):
    station_with_names.append([station[idx],station_kor[idx]])

In [13]:
data_hourly['동'] = ''

In [14]:
###serial기반 동 컬럼 추가
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_hourly_동_v1.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in data_hourly.serial.unique():
    for idx2 in station_with_names:
        if idx in idx2:
            temp = data_hourly[data_hourly['serial']==str(idx)]
            temp['동'] = idx2[1]
            if file.exists() == False:
                temp.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_hourly_동_v1.csv', mode = 'a')
            else:
                temp.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_hourly_동_v1.csv', mode = 'a', header=False)

C:\Users\dty\.conda\envs\June\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
data_hourly = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_hourly_동_v1.csv', index_col = 0)

In [16]:
###동별 시간별 평균값
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_hourly_동_v2.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in data_hourly.동.unique():
    df_mean = data_hourly[data_hourly['동'] == '{}'.format(idx)].groupby('date_hourly').mean().reset_index().copy()
    df_mean['동'] = idx
    if file.exists() == False:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_hourly_동_v2.csv', mode = 'a')
    else:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_hourly_동_v2.csv', mode = 'a', header=False)

In [17]:
data_hourly = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_hourly_동_v2.csv', index_col = 0).reset_index(drop=True)

In [18]:
def date_cuter(dates):
    datelist=[]
    for idx in dates:
        date = str(idx)
        datelist.append(date[:8])
    return datelist

In [19]:
df['date_daily'] = date_cuter(df.tm)
df.index = df.date_daily
df = df.sort_index()
df = df.drop(columns = 'date_daily')

In [20]:
### serial 일별 평균값 
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_daily_v1_08_22.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in df.serial.unique():
    df_mean = df[df['serial'] == '{}'.format(idx)].reset_index().groupby('date_daily').mean().reset_index().copy().drop(columns = 'tm')
    df_mean['serial'] = idx
    if file.exists() == False:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_daily_v1_08_22.csv', mode = 'a')
    else:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_daily_v1_08_22.csv', mode = 'a', header=False)

In [21]:
date_daily = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\종로구_daily_v1_08_22.csv', index_col=0).reset_index(drop=True)

In [22]:
date_daily['동'] = ''

In [23]:
###serial기반 동 컬럼 추가
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_daily_동_v1.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in date_daily.serial.unique():
    for idx2 in station_with_names:
        if idx in idx2:
            temp = date_daily[date_daily['serial']==str(idx)]
            temp['동'] = idx2[1]
            if file.exists() == False:
                temp.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_daily_동_v1.csv', mode = 'a')
            else:
                temp.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_daily_동_v1.csv', mode = 'a', header=False)

C:\Users\dty\.conda\envs\June\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
date_daily = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_daily_동_v1.csv', index_col = 0)

In [25]:
###동별 일별 평균값
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_daily_동_v2.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in date_daily.동.unique():
    df_mean = date_daily[date_daily['동'] == '{}'.format(idx)].groupby('date_daily').mean().reset_index().copy()
    df_mean['동'] = idx
    if file.exists() == False:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_daily_동_v2.csv', mode = 'a')
    else:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_daily_동_v2.csv', mode = 'a', header=False)

In [30]:
date_daily = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_종로_daily_동_v2.csv', index_col = 0).reset_index(drop=True)

In [31]:
date_daily

,date_daily,pm10,noise,temp,humi,pm25,동
0,20180401,67.465830,68.205718,17.522594,60.066946,32.028591,교남동
1,20180402,66.790972,67.615972,20.538472,57.325000,25.959028,교남동
2,20180403,64.900694,67.488889,19.965000,62.589583,21.773611,교남동
3,20180404,12.637955,68.921569,15.465336,59.499300,7.268908,교남동
4,20180405,9.552301,69.341702,9.936541,77.488842,5.541841,교남동
5,20180406,70.600694,70.055556,8.182986,62.739583,17.718056,교남동
6,20180407,36.880056,70.384240,6.536123,36.814505,14.822873,교남동
7,20180408,18.957283,70.811625,6.088305,49.742297,8.766807,교남동
8,20180409,36.793750,70.268750,10.255208,48.647222,22.739583,교남동
9,20180410,38.401130,69.423023,17.110028,42.057203,15.733757,교남동
